In [110]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, TimeDistributed, Dropout, Activation
import yfinance as fyf
from pandas_datareader import data as pdr
fyf.pdr_override() # <-- Here is the fix
import datetime

In [111]:
def get_data(name, start_date, end_date):
    return pdr.get_data_yahoo(name, start_date, end_date)

In [124]:
def prepare_data(data_val, train_data_len, lstm_neuron_number) :
    mm_scalar = MinMaxScaler(feature_range=(0, 1))
    sc_data = mm_scalar.fit_transform(data_val)
    train_data = sc_data[0:train_data_len, :]
    x_train = []
    y_train = []
    trainingdatasize = len(train_data)
    for i in range(lstm_neuron_number, trainingdatasize):
        x_train.append(
            train_data[i - lstm_neuron_number : i, 0]
        )  # contains lstm_neuron_number values 0-lstm_neuron_number
        y_train.append(train_data[i, 0])  # contains all other values

    # Convert the x_train and y_train to numpy arrays
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    test_data = sc_data[train_data_len - lstm_neuron_number:, :]
    x_test = []
    y_test = data_val[train_data_len:, :]
    for i in range(lstm_neuron_number, len(test_data)):
        x_test.append(test_data[i - lstm_neuron_number : i, 0])

    return x_train, y_train, x_test, y_test


In [125]:
def get_lstm_model(lstm_neuron_number, x_train):
    model = Sequential()
    model.add(LSTM(lstm_neuron_number, return_sequences=False,
              input_shape=(x_train.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


In [126]:
stock_names = ["AAPL"]
start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2018, 3, 14)
data = get_data(stock_names, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [127]:
epochs = 12; batch_size = 1; lstm_neuron_number = 110
data_val = data.values
train_data_len = math.ceil(len(data_val) * 0.8)

In [129]:
X_train, y_train, X_test, y_test = prepare_data(data_val, train_data_len, lstm_neuron_number)

X_train.shape, y_train.shape

((130, 110, 1), (130,))

In [133]:
model = get_lstm_model(lstm_neuron_number, X_train)
hist = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/12
130/130 [==============================] - 5s 18ms/step - loss: 0.0256
Epoch 2/12
130/130 [==============================] - 2s 19ms/step - loss: 0.0041
Epoch 3/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0038
Epoch 4/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0037
Epoch 5/12
130/130 [==============================] - 2s 19ms/step - loss: 0.0036
Epoch 6/12
130/130 [==============================] - 2s 19ms/step - loss: 0.0032
Epoch 7/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0029
Epoch 8/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0026
Epoch 9/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0024
Epoch 10/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0023
Epoch 11/12
130/130 [==============================] - 2s 18ms/step - loss: 0.0021
Epoch 12/12
130/130 [==============================] - 2s 19ms/step - loss: 0.0022
